In [1]:
import numpy as np

In [2]:
# helper functions
def get_diagnostic_codes(ic: np.ndarray, sys_id: int = None, off: int = 0) -> list:
    '''Returns diagnostic codes for system ID'''
    ix = 0
    diag_codes = []

    while ix < len(ic):       
        pars = f"{ic[ix]:05}"
        opcode = int(pars[3:])
        
        if opcode == 99:
            break
        
        ic = ic + (ix - len(ic) + 2)*[0] if ix+1 >= len(ic) else ic
        
        par_1 = int(pars[2])
        
        if par_1 == 0:
            par_1_ix = ic[ix+1]
        elif par_1 == 1:
            par_1_ix = ix + 1
        else:
            par_1_ix = off + ic[ix+1]
            
        ic = ic + (par_1_ix - len(ic) + 1)*[0] if par_1_ix >= len(ic) else ic        
            
        if opcode == 3:
            ic[par_1_ix] = sys_id
            ix += 2
        elif opcode == 4:
            diag_codes.append(ic[par_1_ix])
            ix += 2
        elif opcode == 9:
            off += ic[par_1_ix]
            ix += 2
            
        else:
            par_2 = int(pars[1])
            
            if par_2 == 0:
                par_2_ix = ic[ix+2]
            elif par_2 == 1:
                par_2_ix = ix + 2
            else:
                par_2_ix = off + ic[ix+2]
            
            ic = ic + (par_2_ix - len(ic) + 1)*[0] if par_2_ix >= len(ic) else ic        

            if opcode == 5:
                ix = ic[par_2_ix] if ic[par_1_ix] != 0 else ix + 3
            elif opcode == 6:
                ix = ic[par_2_ix] if ic[par_1_ix] == 0 else ix + 3
            
            else:
                par_3 = int(pars[0])
                
                if par_3 == 0:
                    par_3_ix = ic[ix+3]
                elif par_3 == 1:
                    par_3_ix = ix + 3
                else:
                    par_3_ix = off + ic[ix+3]
                
                ic = ic + (par_3_ix - len(ic) + 1)*[0] if par_3_ix >= len(ic) else ic
                
                if opcode == 1: 
                    ic[par_3_ix] = ic[par_1_ix] + ic[par_2_ix]
                if opcode == 2:
                    ic[par_3_ix] = ic[par_1_ix] * ic[par_2_ix]
                if opcode == 7:
                    ic[par_3_ix] = 1 if ic[par_1_ix] < ic[par_2_ix] else 0
                if opcode == 8:
                    ic[par_3_ix] = 1 if ic[par_1_ix] == ic[par_2_ix] else 0
                
                ix += 4
        
    return diag_codes

In [3]:
%cd ../..

/Users/mwtmurphy/projects/advent-of-code


/Users/mwtmurphy/Library/Caches/pypoetry/virtualenvs/advent-of-code-rjxGAg1e-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
with open("data/2019/day_5.txt", "r") as infile:
    intcode = np.array(infile.read().split(","), dtype=int)

diag_codes = get_diagnostic_codes(intcode.copy(), 1)
ans_1 = diag_codes[-1]
diag_codes = get_diagnostic_codes(intcode.copy(), 5)
ans_2 = diag_codes[-1]

print("** Day 5 **")
print(f"Diagnostic code for problem 5.1 = {ans_1}")
print(f"Diagnostic code  for problem 5.2 = {ans_2}", end="\n\n")

** Day 5 **
Diagnostic code for problem 5.1 = 6069343
Diagnostic code  for problem 5.2 = 3188550

